In [248]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import xgboost
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

pd.set_option('display.max_columns', 500)

In [249]:
df_full = pd.read_csv('/Users/paul/Documents/Notebooks/yvw_first_break_months.csv', sep='\t', low_memory=False)
df_full.head(1)

,Asset ID,count,Event Time,Event Date,Class Structure,Class Structure ID,Material,Nominal Size (mm),Soil Class,Soil Description,Grant Description,Pipe Inside Diameter (mm),Pipe Material,Pipe Shape,Construction Date (YYYYMMDD),Pipe Lining,Date Insulated,Tapping Status,Road Name,Road Type,Pipe Length (m),Pipe Status,Height Start of Pipe (meters above sea level),Height End of Pipe (meters above sea level),Latitude Start of Pipe,Lattitude End of Pipe,Longitude Start of Pipe,Longitude End of Pipe,Distribution Zone ID,Water Quality Zone ID,Description,Failure Code,Failure Description,Job Plan,Job Plan Number,Service Location Id,Formatted Address,City,Post Code,Latitude,Longitude,months_break
0,100259,1,15:54:00.0000000,2004-04-05,Repair Burst Water Main,WRBRST,MEDIUM DENSITY POLYETHYLENE,125,EX,Moderate shrink/swell potential.,Clay.,125,MEDIUM DENSITY POLYETHYLENE,CIRCULAR,2003-07-10,NONE,(null),YES,MOUNT DANDENONG TOURIST,ROAD,396.8047,INTEGRITY,500.0,484.19,-37.844721,-37.841505,145.360402,145.358637,1.550536e+09,1.125029e+09,REP LK 1.5M DEEP NON MS 100DIA,UNKN,UNKN,NaN,NaN,1399527.0,1417 MT DANDENONG TOURIST ROAD OLINDA 3788,OLINDA,3788.0,-37.84193818,145.35890900,3


In [250]:
df = df_full[['Class Structure ID', 'Material', 'Nominal Size (mm)', 'Soil Class', 'Grant Description', 'Pipe Inside Diameter (mm)', 'Pipe Material', 'Pipe Lining', 'Pipe Length (m)', 'Height Start of Pipe (meters above sea level)', 'Height End of Pipe (meters above sea level)', 'months_break']]
df.head()
# df[['months_break']].median()

,Class Structure ID,Material,Nominal Size (mm),Soil Class,Grant Description,Pipe Inside Diameter (mm),Pipe Material,Pipe Lining,Pipe Length (m),Height Start of Pipe (meters above sea level),Height End of Pipe (meters above sea level),months_break
0,WRBRST,MEDIUM DENSITY POLYETHYLENE,125,EX,Clay.,125,MEDIUM DENSITY POLYETHYLENE,NONE,396.8047,500.0000,484.19,3
1,WRBRST,"POLYETHYLENE(SOLID WALL, HIGH DENSITY)",125,EX,Clay.,125,"POLYETHYLENE(SOLID WALL, HIGH DENSITY)",NONE,291.2699,50.0000,34.94,7
2,WRBRST,UPVC-THICK WALLED PRESSURE PIPE(INCL BLUE BRUTE),300,VE,Expansive Clay.,300,UPVC-THICK WALLED PRESSURE PIPE(INCL BLUE BRUTE),NONE,147.2552,151.0000,155.00,21
3,WRBRST,UPVC-THICK WALLED PRESSURE PIPE(INCL BLUE BRUTE),100,VE,Expansive Clay.,100,UPVC-THICK WALLED PRESSURE PIPE(INCL BLUE BRUTE),NONE,62.3840,147.6768,147.50,3
4,WRBRST,MEDIUM DENSITY POLYETHYLENE,63,VE,Expansive Clay.,63,MEDIUM DENSITY POLYETHYLENE,NONE,56.0505,114.7660,111.10,17


In [251]:
df = df[df['Pipe Material'] != 'ASBESTOS CEMENT(INCL HARDYFLEX)']
df = df[df['Pipe Material'] != 'DUCTILE IRON CEMENT LINED']
df = df[df['Pipe Material'] != 'GLASS REINFORCED PLASTIC(INCL HOBAS)']
df = df[df['Pipe Material'] != 'DUCTILE IRON(SPHERIOD GRAPHITE CAST IRON)']
df = df[df['Pipe Material'] != 'CAST IRON CEMENT LINED']
df = df[df['Pipe Material'] != 'COPPER']
# df.groupby('Pipe Material')['Asset ID'].nunique()


In [252]:
df = df[df['Soil Class'] != '(null)']
# df.groupby('Soil Class')['Asset ID'].nunique()

In [253]:
# df.groupby('Grant Description')['Asset ID'].nunique()

In [254]:
train_pre, test_pre = train_test_split(df, test_size = 0.3)

In [ ]:
test = pd.DataFrame(columns=test_pre.columns)
train = pd.DataFrame(columns=train_pre.columns)

In [255]:
for index, row in test_pre.iterrows():
    end_month = int(row['months_break'])
    for month in range(1, end_month + 1):
        if month == end_month:
            new_row = row
            new_row['age'] = month
            new_row['will_break'] = True
            test = test.append(new_row)

In [256]:
for index, row in train_pre.iterrows():
    end_month = int(row['months_break'])
    for month in range(1, end_month + 1):
        new_row = row
        new_row['age'] = month
        if month >= end_month - 5:
            new_row['will_break'] = True
        else:
            new_row['will_break'] = False

        train = train.append(new_row)

In [257]:
test

,Class Structure ID,Material,Nominal Size (mm),Soil Class,Grant Description,Pipe Inside Diameter (mm),Pipe Material,Pipe Lining,Pipe Length (m),Height Start of Pipe (meters above sea level),Height End of Pipe (meters above sea level),months_break,age,will_break
1173,WRBRST,UPVC-THICK WALLED PRESSURE PIPE(INCL BLUE BRUTE),100,EX,Clay.,100,UPVC-THICK WALLED PRESSURE PIPE(INCL BLUE BRUTE),NONE,156.1033,67.2659,65.6962,5,5.0,1.0
1505,WRBRST,UPVC-THICK WALLED PRESSURE PIPE(INCL BLUE BRUTE),100,EX,Clay.,100,UPVC-THICK WALLED PRESSURE PIPE(INCL BLUE BRUTE),NONE,162.7821,73.1831,76.9803,1,1.0,1.0
1117,WRBRST,UPVC-THICK WALLED PRESSURE PIPE(INCL BLUE BRUTE),300,VE,Expansive Clay.,300,UPVC-THICK WALLED PRESSURE PIPE(INCL BLUE BRUTE),NONE,39.0162,98.5000,99.4600,35,35.0,1.0
419,WRBRST,"POLYETHYLENE(SOLID WALL, HIGH DENSITY)",180,EX,Clay.,180,"POLYETHYLENE(SOLID WALL, HIGH DENSITY)",NONE,221.3647,132.4100,134.0322,32,32.0,1.0
1744,WRBRST,OPVC - MOLECULAR ORIENTED,100,EX,Clay.,100,OPVC - MOLECULAR ORIENTED,NaN,64.1960,95.7830,90.9630,11,11.0,1.0
1150,WRBRST,UPVC - MODIFIED,100,EX,Clay.,100,UPVC - MODIFIED,NONE,89.2421,298.6909,298.4262,35,35.0,1.0
1975,WRBRST,UPVC - MODIFIED,100,EX,Clay.,100,UPVC - MODIFIED,NONE,185.3523,46.6500,43.4100,36,36.0,1.0
1776,WRBRST,UPVC-THICK WALLED PRESSURE PIPE(INCL BLUE BRUTE),225,EX,Clay.,225,UPVC-THICK WALLED PRESSURE PIPE(INCL BLUE BRUTE),NONE,22.5231,307.7900,309.8917,62,62.0,1.0
1195,WRBRST,PLASTIC,100,EX,Clay.,100,PLASTIC,NONE,391.6927,136.0830,117.8000,8,8.0,1.0
819,WRBRST,UPVC-THICK WALLED PRESSURE PIPE(INCL BLUE BRUTE),100,EX,Clay.,100,UPVC-THICK WALLED PRESSURE PIPE(INCL BLUE BRUTE),NONE,95.8589,100.0000,103.2090,46,46.0,1.0


In [258]:
# test['will_break'] = test['will_break'].astype('str')
test['months_break'] = test['months_break'].astype('int')
# test['will_break'] = test['will_break'].astype('float').astype('int').astype('bool')
test['Nominal Size (mm)'] = test['Nominal Size (mm)'].astype('float')
test['Pipe Inside Diameter (mm)'] = test['Pipe Inside Diameter (mm)'].astype('float')



# train['will_break'] = train['will_break'].astype('str')
train['months_break'] = train['months_break'].astype('int')
# train['will_break'] = train['will_break'].astype('float').astype('int').astype('bool')
train['Nominal Size (mm)'] = train['Nominal Size (mm)'].astype('float')
train['Pipe Inside Diameter (mm)'] = train['Pipe Inside Diameter (mm)'].astype('float')


train.dtypes
# test_ready = pd.get_dummies(test)
# test_ready.

Class Structure ID                                object
Material                                          object
Nominal Size (mm)                                float64
Soil Class                                        object
Grant Description                                 object
Pipe Inside Diameter (mm)                        float64
Pipe Material                                     object
Pipe Lining                                       object
Pipe Length (m)                                  float64
Height Start of Pipe (meters above sea level)    float64
Height End of Pipe (meters above sea level)      float64
months_break                                       int64
age                                              float64
will_break                                       float64
dtype: object

In [259]:
test_ready = pd.get_dummies(test)
train_ready = pd.get_dummies(train)

# test_ready.sort_values('will_break')

In [260]:
test_ready = test_ready.drop('months_break', 1)
train_ready = train_ready.drop('months_break', 1)
train_ready

,Nominal Size (mm),Pipe Inside Diameter (mm),Pipe Length (m),Height Start of Pipe (meters above sea level),Height End of Pipe (meters above sea level),age,will_break,Class Structure ID_WRBRST,Class Structure ID_WRLEAK,Class Structure ID_WRPWSL,Material_MEDIUM DENSITY POLYETHYLENE,Material_MILD STEEL CEMENT LINED,Material_OPVC - MOLECULAR ORIENTED,Material_PLASTIC,"Material_POLYETHYLENE(SOLID WALL, HIGH DENSITY)",Material_UPVC - MODIFIED,Material_UPVC-THICK WALLED PRESSURE PIPE(INCL BLUE BRUTE),Soil Class_EX,Soil Class_SE,Soil Class_ST,Soil Class_VE,Grant Description_Clay.,Grant Description_Expansive Clay.,Grant Description_Sand.,Grant Description_Silty Soil.,Pipe Material_MEDIUM DENSITY POLYETHYLENE,Pipe Material_MILD STEEL CEMENT LINED,Pipe Material_OPVC - MOLECULAR ORIENTED,Pipe Material_PLASTIC,"Pipe Material_POLYETHYLENE(SOLID WALL, HIGH DENSITY)",Pipe Material_UPVC - MODIFIED,Pipe Material_UPVC-THICK WALLED PRESSURE PIPE(INCL BLUE BRUTE),Pipe Lining_NONE
725,150.0,150.0,93.5369,72.1600,70.6100,1.0,0.0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
725,150.0,150.0,93.5369,72.1600,70.6100,2.0,0.0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
725,150.0,150.0,93.5369,72.1600,70.6100,3.0,0.0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
725,150.0,150.0,93.5369,72.1600,70.6100,4.0,0.0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
725,150.0,150.0,93.5369,72.1600,70.6100,5.0,0.0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
725,150.0,150.0,93.5369,72.1600,70.6100,6.0,0.0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
725,150.0,150.0,93.5369,72.1600,70.6100,7.0,0.0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
725,150.0,150.0,93.5369,72.1600,70.6100,8.0,0.0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
725,150.0,150.0,93.5369,72.1600,70.6100,9.0,0.0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
725,150.0,150.0,93.5369,72.1600,70.6100,10.0,1.0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1


In [261]:
y = train_ready.pop('will_break')
y_test = test_ready.pop('will_break')

In [262]:
# gbt = GradientBoostingClassifier(max_depth=8, n_estimators=1000, subsample=.5, random_state=0, learning_rate=.001)
# gbt.fit(train_ready, y)

In [263]:
# cross_val_score(gbt, test_ready, y_test, scoring='accuracy').mean()

In [264]:
# test_ready

In [265]:
# feature_importance = gbt.feature_importances_
# feature_importance = 100.0 * (feature_importance / feature_importance.max())
# features_df = pd.DataFrame({"Features": train_ready.columns, "Importance Score": feature_importance})
# features_df.sort_values("Importance Score", inplace = True, ascending = False)

# features_df.head()
# print(features_df)

In [266]:
# cross_val_score(gbt, sanity_pre_train_ready, sanity_y, scoring='accuracy').mean()

In [267]:
# cross_val_score(gbt, sanity_pre_train_ready, sanity_y, scoring='accuracy').mean()

In [268]:
xg = xgboost.XGBClassifier(min_child_weight=.2, subsample=.2, n_estimators=15000, max_depth=10,colsample_bytree=.2)
xg.fit(train_ready, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.2, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=10, min_child_weight=0.2, missing=None,
       n_estimators=15000, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.2)

In [269]:
test_ready

,Nominal Size (mm),Pipe Inside Diameter (mm),Pipe Length (m),Height Start of Pipe (meters above sea level),Height End of Pipe (meters above sea level),age,Class Structure ID_WRBRST,Class Structure ID_WRLEAK,Class Structure ID_WRPWSL,Material_MEDIUM DENSITY POLYETHYLENE,Material_MILD STEEL CEMENT LINED,Material_OPVC - MOLECULAR ORIENTED,Material_PLASTIC,"Material_POLYETHYLENE(SOLID WALL, HIGH DENSITY)",Material_UPVC - MODIFIED,Material_UPVC-THICK WALLED PRESSURE PIPE(INCL BLUE BRUTE),Soil Class_EX,Soil Class_SE,Soil Class_ST,Soil Class_VE,Grant Description_Clay.,Grant Description_Expansive Clay.,Grant Description_Sand.,Grant Description_Silty Soil.,Pipe Material_MEDIUM DENSITY POLYETHYLENE,Pipe Material_MILD STEEL CEMENT LINED,Pipe Material_OPVC - MOLECULAR ORIENTED,Pipe Material_PLASTIC,"Pipe Material_POLYETHYLENE(SOLID WALL, HIGH DENSITY)",Pipe Material_UPVC - MODIFIED,Pipe Material_UPVC-THICK WALLED PRESSURE PIPE(INCL BLUE BRUTE),Pipe Lining_NONE
1173,100.0,100.0,156.1033,67.2659,65.6962,5.0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
1505,100.0,100.0,162.7821,73.1831,76.9803,1.0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
1117,300.0,300.0,39.0162,98.5000,99.4600,35.0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,1
419,180.0,180.0,221.3647,132.4100,134.0322,32.0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1
1744,100.0,100.0,64.1960,95.7830,90.9630,11.0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
1150,100.0,100.0,89.2421,298.6909,298.4262,35.0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1
1975,100.0,100.0,185.3523,46.6500,43.4100,36.0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1
1776,225.0,225.0,22.5231,307.7900,309.8917,62.0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
1195,100.0,100.0,391.6927,136.0830,117.8000,8.0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1
819,100.0,100.0,95.8589,100.0000,103.2090,46.0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1


In [270]:
y_pred = xg.predict(test_ready)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 85.60%


In [271]:
feature_importance = xg.feature_importances_
feature_importance = 100.0 * (feature_importance / feature_importance.max())
features_df = pd.DataFrame({"Features": train_ready.columns, "Importance Score": feature_importance})
features_df.sort_values("Importance Score", inplace = True, ascending = False)

features_df.head()
print(features_df)


                                             Features  Importance Score
2                                     Pipe Length (m)        100.000000
3       Height Start of Pipe (meters above sea level)         96.735840
4         Height End of Pipe (meters above sea level)         95.410812
5                                                 age         76.018204
0                                   Nominal Size (mm)         20.934853
1                           Pipe Inside Diameter (mm)         17.936337
16                                      Soil Class_EX          6.562251
15  Material_UPVC-THICK WALLED PRESSURE PIPE(INCL ...          5.993411
19                                      Soil Class_VE          5.970332
13    Material_POLYETHYLENE(SOLID WALL, HIGH DENSITY)          5.389725
20                            Grant Description_Clay.          5.131326
30  Pipe Material_UPVC-THICK WALLED PRESSURE PIPE(...          5.104174
21                  Grant Description_Expansive Clay.          5